# Recurrent Neural Network - Recap

👋 Welcome to this recap of the RNN unit!

📆 We will be revisiting *Challenge 3 - Predict Weather Temperature*.


🎯 ***Predict one target (`N_TARGETS = 1`)***: 
- 🌡 `T (degC)` = temperature in Celsius degrees


We will introduce two modifications to this challenge:

* (1) 🆕 an `OUTPUT_LENGTH > 1`
    * $ \Leftrightarrow$ we will try to predict <u>***MULTIPLE time steps in the future***</u>
    
    

* (2) 🆕 a `HORIZON ≥1` between the last seen value in $X$ and the first value $y$ to predict in the future:
    * $ \Leftrightarrow$ we want to <u>***make the predictions a few moments AFTER the last known values***</u> 

## (0) Libraries

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Deep Learning
import tensorflow as tf

# Checking data types
from typing import Dict, List, Tuple, Sequence
import string

## (1) 📚 The weather dataset

### (1.1) Loading the dataset

🌦 For this recap, we will re-use the preprocessed weather dataset from the *challenge 3 - Predict Weather Temperature*. 


💾 Load it.

In [ ]:
url = "https://wagon-public-datasets.s3.amazonaws.com/deep_learning_datasets/weather_every_three_hours_engineered.csv"
df = pd.read_csv(url).drop(columns = ['Unnamed: 0'])
display(df.tail())
print(f"df.shape = {df.shape}")

👆 *(Reminders*) This weather dataset is a ***single Time Series*** represented as a DataFrame, i.e. a **2D-array**.
- `df.shape = (n_timesteps, n_features)`
    - `n_timesteps` $= 23$k rows  (_~8 years of weather data, from 2009 to 2016 with records every 3 hours_)
    - `n_features` $= 19$ features composed of:
        - $1$ <font color=green>**target**</font> (we will use the past values of the temperature as a feature)
        - $18$ <font color=orange>**past covariates**</font> (= features which past values are known)
        - $0$ <font color=blue>**future covariates**</font> (= features which future values are known, e.g. public holidays)
    
<img src='https://github.com/lewagon/data-images/blob/master/DL/time-series-covariates.png?raw=true'>    

### (1.2) 🌅 The big picture about dealing with Time Series *(reminder)*

1️⃣ <b>[FOLDS] <u>Cross-Validation in Time Series  </u></b>

Starting from this single Time Series:
- We will create <font color="#c91ac9">**FOLDS**</font>
- <font color=blue>**Train**</font>/<font color="#ff8005">**Evaluate**</font> our LSTM  <font color="#c91ac9">**on each of these different FOLDS**</font> to conclude about <b><u>the robustness of the model</u><b>.
    
_It is very common to create hundreds of folds in Time Series forecasting, in order to cover all types of external conditions: crash market periods, bull markets, atone markets, etc..._

2️⃣ <b>[TRAIN-TEST SPLIT] <u>Holdout method</u></b>

For each <font color="#c91ac9">**FOLD**</font>, we will do a <font color=blue>**TRAIN**</font>-<font color="#ff8005">**TEST**</font> SPLIT to:
* <font color=blue>**fit**</font> the model on the <font color=blue>**train**</font> set 
* <font color="#ff8005">**evaluate**</font> it on the <font color="#ff8005">**test**</font> set

_Always split the train set **chronologically** before the test set!_

<hr> 

👇 The first two steps can be summarized in the following image (here, we illustrated a 4-fold temporal cross-validation):

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/time_series_cross_validation.png" alt="Time Series Cross Validation" width="800" height="400">

<hr>

3️⃣ <b>[SEQUENCES] <u>Sampling/Extracting sequences</u></b>


After splitting each fold into a <font color="blue">train</font> set and a <font color="#ff8005">test</font> set, it is time to:
- 🏋 sample lots of <font color="#884dff"><i>sequences</i></font> $\color{blue}{(X_i, y_i)}$ on which the model will be <font color="blue">trained</font>
- 👩🏻‍🏫 sample lots of <font color="#884dff"><i>sequences</i></font> $\color{#ff8005}{(X_i, y_i)}$ on which the model will be <font color="#ff8005">evaluated</font>


<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="500" height="500"> 

👉 All these <font color="#884dff"><i>sequences</i></font> in the <font color="blue">train</font> set and the <font color="#ff8005">test</font> set will have a common shape `(input_length, n_features)` $ = (14\times8,19) = (112,19)$.

👉 Each <font color="#884dff"><i>sequence</i></font> has a target, the shape of which will be `(output_length, n_targets)` $ = (7\times8, 1) = (56, 1)$.

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/shapes_for_input_sequence_and_ouput_sequence.png" alt="3d arrays time series" width="1200" height="800"> 

<hr>

> 🔥 Open this [**infograph**](https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/big_picture_temporal_data_handling.png)  side-by-side with the notebook for a visual summary! 🔥

## (2) Adapting the functions from the _"Predict Temperature"_ challenge

### (2.0) Global Variables

🌐 Let's define some ***global variables*** that we will use for our tests everywhere in this notebook:

In [ ]:
# Folds
FOLD_LENGTH = 3*365*8      # each fold will have 3 years of data
FOLD_STRIDE = int(365/2)*8 # sliding each semester

# Temporal Train-Test split
TRAIN_TEST_RATIO = 0.66    
N_TRAIN = 6666 # number_of_sequences_train for each fold_train
N_TEST =  3333 # number_of_sequences_test for each fold_test

# Inputs
N_FEATURES = 19
INPUT_LENGTH = 14*8 # - Records every 3 hours x 8 = 24 hours 
                    # - During two weeks, which is quite common for weather forecasts  

# Outputs
TARGET = ['T (degC)']
TARGET_COLUMN_IDX = 1 # 'T (degC)' corresponds to the second column of the df
N_TARGETS = 1
OUTPUT_LENGTH = N_TARGETS*7*8 # - Predicting one target, the temperature 
                              # - for one week with predictions every 3 hours

# Additional parameters
HORIZON = 7*8 # - We are predicting the week after the next week

### (2.1)  🗂 <font color="#c91ac9">FOLDS</font>

🎁 **`get_folds`**

You have already coded this function in *Challenge 3 - Predict Weather Temperature*. <br> _Feel free to ask the teacher any questions if something is still not clear._

In [ ]:
def get_folds(df: pd.DataFrame, 
              fold_length: int,
              fold_stride: int) -> List[pd.DataFrame]:
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold
    
    Returns a list of folds, each as a DataFrame
    '''

    folds = []
    for idx in range(0, len(df), fold_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]
        folds.append(fold)
    return folds

👉 Let's generate these <font color="#c91ac9">**FOLDS**</font>.

In [ ]:
folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)

print(f'The function generated {len(folds)} folds.')
print(f'Each fold has a shape equal to {folds[0].shape}.')

🧪 Make sure that the following <font color=green>***assert***</font> doesn't return anything (which means the assertion is correct).

In [ ]:
assert(folds[0].shape == (8760, 19))

☝️ This amount of <font color="#c91ac9">**FOLDS**</font> should be enough to cross-validate our model correctly!

### (2.2) ✂️  Temporal <font color=blue>Train</font>/<font color="#ff8005">Test</font> Split

👩🏻‍🏫 Let's <font color="#c91ac9">focus on one fold</font> for the moment, the first one for example.

In [ ]:
fold = folds[0]
fold

⌚️ We want to ***split this <font color="#c91ac9">fold</font> chronologically*** into a <font color=blue>***fold_train***</font> and a <font color="#ff8005">***fold_test***</font>.

*Each of these fold_train and fold_test will contain all the data we need to be able to sample many pairs `(Xi, yi)` in a next step!*

#### (2.2.1) 🧑🏻‍🏫 The complexity introduced by the <font color=green>gap</font>

<details>
    <summary>⏸ Let's take a <i>quick break</i> from this weather dataset to understand what is a temporal gap.</summary>

<hr>

🤑 Put yourself in the shoes of a quantitative trader at timesteps $\color{blue}{1}, \color{blue}{2}, \color{blue}{3}, ..., \color{blue}{10}$.

Let's say you are in the <font color=blue>training</font> phase, and you want to stop your training after day `10`:
- You have to wait until day `10` to know the real value $y_{10}$, and compare it with the predicted value $\hat{y}_{10}$ to train the model.
 
Here are the <font color=blue>assumptions about your model</font>:
- It is trained on <font color="#884dff"><i>sequences</i></font> with `INPUT_LENGTH = 3`
- The goal is to predict `OUTPUT_LENGTH = 1` point in the future
- You want to predict this point `HORIZON = 4` days after the last known value.

✅ Imagine that your model was trained and put into production

❓ <u>On which day can you evaluate the live performance of the model for the first time</u> ❓

* You are at day `10`. Hence, the first prediction you can make is for day `10` + `HORIZON` = `14`
* You will have to wait until day `14` to see how good was your prediction!
    * `y_test_first` = day `14`
* You sit IDLE for <font color=green>3</font> days $(11, 12, 13)$
    
</details>    

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/train_test_split_with_horizon.jpg?raw=true" height=500 width=800>

👉 The <font color=green>gap</font> between the <font color=blue>train</font> and the <font color=orange>test</font> set should be equal to `GAP = HORIZON - 1`.

#### (2.2.2) 💻 Adapting the  `train_test_split` function accordingly

💻 ❓ **Question ❓ (Temporal <font color=blue>Train</font>/<font color=orange>Test</font> split)** 

Code the function `train_test_split` down below which:
- <i>(inputs)</i> given 
    - a `fold` (like above), 
    - a `train_test_ratio` (e.g 0.8) 
    - an `input_length` (fixed)
    - 🆕 a `horizon` (fixed)
- <i>(output)</i> returns a tuple (`fold_train`, `fold_test`) of DataFrames

In [ ]:
def train_test_split(fold: pd.DataFrame, 
                     train_test_ratio: float,
                     input_length: int, 
                     horizon: int) -> Tuple[pd.DataFrame]:
    '''
    Returns a train dataframe and a test dataframe (fold_train, fold_test)
    from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))   
    '''
    pass  # YOUR CODE HERE

💻 ❓ **Question ❓ (Temporal <font color=blue>Train</font>/<font color=orange>Test</font> split for <font color="#c91ac9">one fold </font>)** 

Split the <font color="#c91ac9">fold #0</font>.


<details>
    <summary><i>Reminders</i></summary>

*As a reminder, in  section `(2.0) Global Variables`, we defined*:
- *`TRAIN_TEST_RATIO` = 66%* 
- *`INPUT_LENGTH` = 2 weeks = 112 time steps for each `Xi`, which is quite common in weather forecasting*
- *`HORIZON` = 1 day = 8 time steps*

</details>


In [ ]:
# YOUR CODE HERE

🧪 Check that your shapes and your starting/ending points are correct:

In [ ]:
## Double-checking the shapes for the fold_train and fold_test
fold_train.shape == (5782, 19)
fold_test.shape == (3090, 19)

## Double-checking the starting point and the ending point for both folds
assert (fold_train.index.start) == 0
assert (fold_train.index.stop) == 5782
assert (fold_test.index.start) == 5670
assert (fold_test.index.stop) == 8760

### (2.3) 💻 🔢 Create (X, y) sequences

In [ ]:
# Inputs
print("##### INPUTS #####")
print(f'- INPUT_LENGTH = {INPUT_LENGTH} timesteps = {int(INPUT_LENGTH/8)} days = {int(INPUT_LENGTH/8/7)} weeks')
print(f'- N_FEATURES = {N_FEATURES}')
# Outputs
print("##### OUTPUTS #####")
print(f'- OUTPUT_LENGTH = {OUTPUT_LENGTH} timesteps = {int(OUTPUT_LENGTH/8)} day(s)')
print(f'- N_TARGETS = {N_TARGETS}')
# Parameters
print("##### PARAMETERS #####")
print(f'- HORIZON = {HORIZON} timesteps = {int(HORIZON/8)} day(s)')
# Train
print("##### TRAIN SET #####")
print(f"- The training fold starts at index {fold_train.index.start} and stops at index {fold_train.index.stop}.")
# Test
print("##### TEST SET #####")
print(f"- The test fold starts at index {fold_test.index.start} and stops at index {fold_test.index.stop}.")

In [ ]:
# New: Scanning  through a fold  
STRIDE = 3*8 # sliding every three days, for instance
print(f'STRIDE = {STRIDE} timesteps = {int(STRIDE/8)} day(s)')

💻 ❓ **Question ❓ (Extracting a <font color="#884dff"><i>sequence</i></font>)** 

Code the function `get_Xi_yi` which extracts a <font color="#884dff"><i>sequence</i></font> from a Time Series: it should take the following arguments:
- `first_index`
- `data` (your 2D-dataframe representing the Time Series)
- `input_length`
- `output_length`
- `horizon`

In [ ]:
def get_Xi_yi(first_index: int, 
              fold: pd.DataFrame, 
              horizon: int,
              input_length: int,
              output_length: int) -> Tuple[np.ndarray, np.ndarray]:
    '''
    - extracts one sequence from a fold
    - returns a pair (Xi, yi) with:
        * len(Xi) = `input_length` and Xi starting at first_index
        * len(yi) = `output_length`
        * last_Xi and first_yi separated by the gap = horizon -1
    '''

    pass  # YOUR CODE HERE

🧪 Run the following cell to test your function:

In [ ]:
## Testing your function get_Xi_yi

first_index = fold.index.start
Xi, yi = get_Xi_yi(first_index=first_index,
                   fold=fold,
                   horizon=HORIZON,
                   input_length=INPUT_LENGTH,
                   output_length=OUTPUT_LENGTH)

assert (Xi.index.start == first_index)
assert (Xi.shape == (INPUT_LENGTH, 19))
assert (yi.index.stop == Xi.index.stop + HORIZON - 1 + OUTPUT_LENGTH)
assert (yi.shape == (OUTPUT_LENGTH, 1))

💻 ❓ **Question ❓ (Creating <font color="#884dff"><i>sequences</i></font>, scanning chronologically through a fold)** 

Code the function `get_X_y` to scan an entire fold and extract sequences.

In [ ]:
def get_X_y(fold: pd.DataFrame,
            horizon: int,
            input_length: int,
            output_length: int,
            stride: int,
            shuffle=True) -> Tuple[np.ndarray, np.ndarray]:
    """
    - Uses `data`, a 2D-array with axis=0 for timesteps, and axis=1 for (targets+covariates columns)
    - Returns a Tuple (X,y) of two ndarrays :
        * X.shape = (n_samples, input_length, n_covariates)
        * y.shape =
            (n_samples, output_length, n_targets) if all 3-dimensions are of size > 1
            (n_samples, output_length) if n_targets == 1
            (n_samples, n_targets) if output_length == 1
            (n_samples, ) if both n_targets and lenghts == 1
    - You can shuffle the pairs (Xi,yi) of your fold
    """

    pass  # YOUR CODE HERE

💻 ❓ **Question ❓ Generate <font color="#884dff"><i>sequences</i></font>** both in the <font color=blue>**train**</font> set and the <font color="#ff8005">**test**</font> set.

In [ ]:
# YOUR CODE HERE

🧪 Test that your shapes are correct. If not, go back to the function to debug it.

In [ ]:
## ASSERTS
assert(X_train.shape == (232, INPUT_LENGTH, N_FEATURES))
assert(y_train.shape == (232, OUTPUT_LENGTH))
assert(X_test.shape == (120, INPUT_LENGTH, N_FEATURES))
assert(y_test.shape == (120, OUTPUT_LENGTH))

## (3) 💻 Modelling

### (3.1) 💻 LSTM

💻 ❓ **Question (the LSTM model)** ❓ (<i>Adapting the code from Challenge 3</i>)

We took the architecture of the LSTM model from the previous challenge. The function `init_model` initializes and compiles it.

👉 Now that we are ***predicting multiple steps in the future***, what do you need to change in this function?

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import regularizers


def init_model(X_train, y_train):

    # 0 - Normalization
    # ======================    
    normalizer = Normalization()
    normalizer.adapt(X_train)
    
    # 1 - RNN architecture
    # ======================    
    model = models.Sequential()    
    # Normalizing Inputs
    model.add(normalizer)
    # Recurrent Layer
    model.add(layers.LSTM(units=64, activation='tanh', return_sequences=False, 
                          recurrent_dropout=0.5, dropout=0.5))
    # Hidden Dense Layer that we are regularizing
    reg_l2 = regularizers.L2(0.5)
    model.add(layers.Dense(32, activation="relu", kernel_regularizer = reg_l2))
    model.add(layers.Dropout(rate=0.5))
    
    # Predictive Dense Layer
    ### model.add(layers.Dense(1, activation='linear'))
    ### QUESTION: HOW DO YOU CHANGE THIS PART FOR MULTIPLE STEPS ?    
    pass  # YOUR CODE HERE
    
    # 2 - Compiler
    # ======================
    initial_learning_rate = 0.01

    lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=1000, decay_rate=0.5)

    adam = optimizers.Adam(learning_rate=lr_schedule)
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])

    return model
    

🎁 **Training**: We can re-use the function `fit_model` from the last challenge to <font color=blue>train</font> the model.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


def fit_model(model: tf.keras.Model, verbose=1) -> Tuple[tf.keras.Model, dict]:

    es = EarlyStopping(monitor="val_loss",
                       patience=10,
                       mode="min",
                       restore_best_weights=True)

    history = model.fit(X_train,
                        y_train,
                        validation_split=0.3,
                        shuffle=False,
                        batch_size=32,
                        epochs=200,
                        callbacks=[es],
                        verbose=verbose)

    return model, history

🎁 **Visualizing performance:** Feel free to use the `plot_history` function to visualize how your model learned.

In [ ]:
def plot_history(history):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # --- LOSS: MSE --- 
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Validation'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- METRICS:MAE ---
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Validation'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax

💻 ❓ **Questions ❓ (Training and Evaluating the LSTM model)**

Using your functions `init_model` and `fit_model`:
1. **Initialize** your model
2. <font color=blue>**Train**</font> it and observe the performances on the train set and the validation set
3. <font color=orange>**Evaluate**</font> it on the test set

_Feel free to apply regularization techniques in the architecture if the model overfits._

In [ ]:
# YOUR CODE HERE

### (3.2) 🎁 Baseline with a horizon

👉 (*Reminder*) ***In Time Series, an "intuitive" baseline model is to repeat the last seen value as (a) prediction(s) for the future value(s)*** you want to forecast, as illustrated down below!

<img src = "https://github.com/davydw/public-pictures/blob/main/last_seen_value_with_horizon.png?raw=true" width = 600 height = 300>

🎁 **The Last Seen Value Baseline Model** 

Let's go together through the function `last_seen_value_baseline` which:
- (_input_) takes a pair $(X, y)$ 
- (_output_) computes the MAE of the `"repeated last seen value"`  baseline model

In [ ]:
TARGET_COLUMN_IDX = 1 # the second column of each fold corresponds to the temperature

def last_seen_value_baseline(X, y):

    # How many values do you want to predict in the future ?
    output_length = y.shape[-1]
    
    # For each sequence, let's consider the last seen value
    # and only the temperature column
    last_seen_values = X[:,-1, TARGET_COLUMN_IDX].reshape(-1,1)

    # We need to duplicate these values as many times as output_length
    # The author of this notebook did not know how to do it, so they searched on Stackoverflow
    # and found this nice np.repeat in Numpy, which is self-explanatory
    repeated = np.repeat(last_seen_values, axis = 1, repeats = output_length)

    return np.mean(np.abs(y_test - repeated))

🕵🏻‍♀️ What is the performance of the "last seen value baseline model" on the <font color=orange>**test**</font> set?

In [ ]:
# 1 - Evaluation of the Baseline Model
# ====================================
mae_baseline = last_seen_value_baseline(X_test, y_test)
print(f"- The Baseline MAE on the test set is equal to {round(mae_baseline,2)} Celsius degrees")

# 4 - Comparison with the LSTM model
# ====================================
print(f"- The LSTM MAE on the test set is equal to {round(res[1],2)} Celsius degrees")
print(f"👉 Improvement/decrease of the LSTM model over the baseline (on this fold for the test set) = : {round((1 - (res[1]/mae_baseline))*100,2)} % 👈")

<u><b><i>Warnings:</i></b></u>

* Do you remember the challenge `Predict weather temperature` where we were trying to predict the temperature in three hours, i.e. ***the next data point***? We were able to improve the MAE by 30-40% on the LSTM vs. the baseline for every fold, and the MAE was more or less between 1 and 2 degrees Celsius.

* <font color=red>***The more data points in the future we try to predict, the higher your MAE will be!***</font>

* *It is more and more frequent to see a 4/5 Celsius degrees - difference between the real temperature in seven days and the currently predicted value!*

### (3.3) 💻 Cross-Validation

ℹ️ Reminders of the global variables in this notebook:

In [ ]:
# Folds
print('##### FOLDS ##### ')
print(f'- FOLD_LENGTH = {FOLD_LENGTH} timesteps = {3} years x {365} days x {8} records per day')
print(f'- FOLD_STRIDE = {FOLD_STRIDE} timesteps = {(365/2)} days x {8} records per day = sliding each semester')

# Chronological Train Test Split
print('##### TRAIN TEST SPLIT #####')
print(f'- TRAIN_TEST_RATIO = {TRAIN_TEST_RATIO}')
print(f'- N_TRAIN = {N_TRAIN}')
print(f'- N_TEST = {N_TEST}')

# Inputs
print('##### INPUTS #####')
print(f'- INPUT_LENGTH = {INPUT_LENGTH} timesteps = {int(INPUT_LENGTH/8)} days x {8} records per day = {int(INPUT_LENGTH/8/7)} weeks')
print(f'- N_FEATURES = {N_FEATURES}') 

# Outputs
print('##### OUTPUTS #####')
print(f'- OUTPUT_LENGTH = {OUTPUT_LENGTH} timesteps = {int(OUTPUT_LENGTH/8)} day(s) x {8} records per day')
print(f"- Trying to predict:{TARGET}")
print(f'- N_TARGETS = {N_TARGETS}') 

# Parameters
print('##### PARAMETERS #####')
print(f'- HORIZON = {HORIZON} timesteps = {int(HORIZON/8)} day(s) x {8} records per day')


In [ ]:
# Remember how many folds do we have ?
folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)
print(f"WARNING, we have {len(folds)} FOLDS, it may take a long time to run...!")

🎁 **Cross-validating the LSTM and the baseline** 

_Let's go together through the following code_ 👇

In [ ]:
# Let's reload the dataset, just in case
url = "https://wagon-public-datasets.s3.amazonaws.com/deep_learning_datasets/weather_every_three_hours_engineered.csv"
df = pd.read_csv(url).drop(columns = ['Unnamed: 0'])

In [ ]:
list_of_mae_baseline_model = []
list_of_mae_recurrent_model = []
    
# 1 - Creating FOLDS
# =======================================================
    
folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)
    
for fold_id, fold in enumerate(folds):    
    
    # 2 - CHRONOLOGICAL TRAIN TEST SPLIT of the current FOLD
    # =======================================================    
    
    (fold_train, fold_test) = train_test_split(fold = fold, 
                                               train_test_ratio = TRAIN_TEST_RATIO, 
                                               input_length = INPUT_LENGTH, 
                                               horizon = HORIZON) 
    
    # 3 - Scanninng fold_train and fold_test for SEQUENCES 
    # =======================================================       
    
    X_train, y_train = get_X_y(fold = fold_train, 
                               horizon = HORIZON, 
                               input_length = INPUT_LENGTH, 
                               output_length = OUTPUT_LENGTH, 
                               stride = STRIDE)
    
    X_test, y_test = get_X_y(fold_test, 
                             horizon = HORIZON, 
                             input_length = INPUT_LENGTH, 
                             output_length = OUTPUT_LENGTH,
                             stride = STRIDE)
    
    # 4.1 - Baseline Model
    # =======================================================
    mae_baseline = last_seen_value_baseline(X_test, y_test)
    list_of_mae_baseline_model.append(mae_baseline)
    
    print("-"*50)
    print(f"MAE baseline fold n°{fold_id} = {round(mae_baseline, 2)}")        
    
    
    # 4.2 - LSTM Model
    # =======================================================
    
    # Initializing the LSTM Model
    model = init_model(X_train, y_train)
    # Training
    model, history = fit_model(model, verbose=0)
    # Evaluation
    res = model.evaluate(X_test, y_test, verbose = 0)
    mae_lstm = res[1]
    
    list_of_mae_recurrent_model.append(mae_lstm)
    
    print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")
    
    # 4.3 - Comparison LSTM vs Baseline for the current fold
    # =======================================================
    print(f"🏋🏽‍♂️ Improvement/Decrease vs. Baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")      

In [ ]:
cv_mae_baseline = np.mean(list_of_mae_baseline_model)
cv_mae_lstm = np.mean(list_of_mae_recurrent_model)

print("-"*50)
print(f"Average MAE baseline = {round(cv_mae_baseline, 2)}")    
print(f"Average LSTM baseline = {round(cv_mae_lstm, 2)}")    
print(f"🏋🏽‍♂️ Improvement/Decrease vs. Baseline: {round((1 - (cv_mae_lstm/cv_mae_baseline))*100,2)} % \n")      

🏁 Congratulations for finishing this day about RNN!

🃏 Don't forget your flashcards!

## 🏁 Final words

* 🤯 Predicting the temperature for 7 days every 3 hours, after waiting for 7 days, is not an easy task at all.
    - The LSTM model did not perform well on some folds
    - 5 degrees is a high MAE, one should take into account how "volatile" the temperature has been over the last two decades...

<hr>


* 🕵🏻 We may need weather data from other cities.
* 🤔 If you plan to use Recurrent Neural Networks during your project, ask your Batch Mananger and the most experienced teachers and TAs if about the quality of your data and the feasibility of your projects.
* 😇 Keep the morale high, RNNs are complex networks but they are pretty useful for _Natural Language Processing_. Stay tuned!

<hr>

#### Bonus: what is the main difficulty when trying to regress multiple targets in RNN ? 

- *Optional question only for very advanced students!*
- *For 99% of the students: read it later :) it's time to go home!*

🕵🏻‍♂️ We could have also tried to predict to <b>predict multiple targets</b>... which would have complexified the problem! 

<hr>

<details>
        <summary><i>Why, you ask ?</i></summary>
    
Imagine that you want to predict `N_TARGETS = 3` for 7 days in the future `OUTPUT_LENGTH = 7 x 8  = 56 timesteps`
    
- 🌡 `T (degC)` = temperature in Celsius degrees
- ♒️ `p (mbar)` = atmospheric air pressure in millibars
- 💦 `rh (%)` = relative humidity expressed as a percent 
    
👍 The predictive Dense Layer in your architecture will contain `3 x (7 x 8) = 168` neurons, that is totally doable. 
    
☢️ However when you compile the model, you will choose `metrics = ['mae']` and will evaluate your model on a metric which doesn't make any sense because you are mixing absolute errors of temperature, air pressure and relative humidity... ***You should create a custom metrics which computes one mae per target*** for example. It is possible  but for this recap, we will stay focused on one feature. Feel free to think about this question if your data science project needs it! </details>        